In [83]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
import os
with open('dataset_raw/openai.api.key.2', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [85]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

#llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
#set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [86]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [87]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                #table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [88]:
def gen_code3(llm, messages, question, value_list): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": value_list, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(value_list = value_list, question = question), code)

def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')
            

In [89]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0, top_p = 1.0, seed=11)
llm = ChatOpenAI(model="gpt-3.5-turbo", 
temperature=0,
    #max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    seed=123)

In [90]:
 messages=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a list of annotated values. You will receive the financial report as an annotated value list and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a list of annotated values: {value_list}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
          "Generate a Python function 'run(value_list)' that can answer the question using the list of annotated values! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            "To calculate the difference, use absolute value. " +
            "To calculate proportion, do not calculate percentage. " +
            "Use all given year values if no year specified. " +
            "The code must be specific to the provided value list. " +
            "Do not generate explanation, nor example code, just the function. "              
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]     


In [95]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

#qid = 'ba6783f3-8207-419a-b407-3f688682caef'
#qid = 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'

#qid = 'bf7abd62-d9cd-48d2-8826-1457684019a3'
#qid = '4d259081-6da6-44bd-8830-e4de0031744c'
#qid = 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'

#qid = '7cd3aedf-1291-4fea-bc9d-a25c65727b7b'
qid = '22e20f25-669a-46b9-8779-2768ba391955' #- difference abs
#qid = '65ec782c-691e-45df-b541-caecb85154ff'
#qid = 'a983501d-2eec-486d-9661-e520c7c8af5e'
#qid = 'a81f1322-e74f-4e3c-a6cf-4b8d25d01cf5'
#qid = '64c902c6-f426-4432-84b3-c10b3065716f'
#qid = '6100c476-160a-4f1e-bfc1-a16f4cc18b52'
#qid = 'ef274d2b-fbd6-4e9f-95f9-79b37827d91c'
#qid = '94916f79-4b6d-487f-8fa2-39f81a085760'
#qid = '72325ec6-41ad-4648-9798-b22a61122cb4'
#qid = '263d03ec-83d2-48df-8376-1a72167798f7'
#qid = '4f7c8e0a-6ae9-40ee-bd16-06db6f96eaf1'
#qid = '35d602ae-9131-4291-a30c-49a40f32bbe4'

qid = 'e9b75381-50b1-4717-a322-31ed1eef4259'
_table, _q =  get_question(qid)
table = _table['table']
print("tid: ",_table['uid'])
[print(r) for r in table]

q = _q['question']
print(q)
values = table_convert.convert_multitable(table)
[print(v) for v in values]
#response = chain.invoke({"value_list": values, "question":q})
#code =  response.replace('```python','').replace('```','')
p, code = gen_code3(llm, messages, q, values)
print(code)
#print(len(p) + len(code))
(v, s) = exec_code(code, values)
rng = 0.01
if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
    print("\033[92m Success: " + str(v)+'\033[0m')
else:    
    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )  

tid:  239347c1-faf9-4fb1-a167-d3c83fd0dead
['', '', 'Year ended March 31,', '']
['', '2019', '2018', '2017']
['Global (India and International)', '', '', '']
['Hindi films', '7', '10', '8']
['Regional films (excluding Tamil films)', '49', '3', '12']
['Tamil films', '3', '1', '3']
['International Only', '', '', '']
['Hindi films', '7', '1', '3']
['Regional films (excluding Tamil films)', '—', '—', '—']
['Tamil films', '—', '—', '12']
['India Only', '', '', '']
['Hindi films', '1', '3', '1']
['Regional films (excluding Tamil films)', '5', '6', '5']
['Tamil films', '—', '0', '1']
['Total', '72', '24', '45']
What is the global increase / (decrease) in the Tamil films from 2017 to 2018?
{'number_value': 7.0, 'scale': '', 'category': 'Hindi films', 'header1': 'Global (India and International)', 'header2': '2019', 'header3': 'Year ended March 31,'}
{'number_value': 10.0, 'scale': '', 'category': 'Hindi films', 'header1': 'Global (India and International)', 'header2': '2018', 'header3': 'Year 

In [99]:
qids = ['010df393-a62e-408d-888a-045a4e435e6a',
'73693527-ed4b-4d07-941e-0e654095a43d',
'426e47ae-aff7-43e2-81be-0bb951b0def1',
'b8afd327-b836-4276-8556-eac89746b09f',
'6ba4045d-4222-4da0-a126-94b267a707ec',
'e9b75381-50b1-4717-a322-31ed1eef4259']
for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    #print("tid: ",_table['uid'])
    #[print(r) for r in table]

    q = _q['question']
    #print(q)
    print(_q['uid'])
    values = table_convert.convert_multitable(table)
    #[print(v) for v in values]
    p, code = gen_code3(llm, messages, q, values)
    #print(len(p) + len(code))
    (v, s) = exec_code(code, values)
    rng = 0.01
    if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 

010df393-a62e-408d-888a-045a4e435e6a
 failure: -14212.0 good answer:  14212 
73693527-ed4b-4d07-941e-0e654095a43d
 failure: 0.02 good answer:  0.9500000000000001 
426e47ae-aff7-43e2-81be-0bb951b0def1
 failure: 259.8 good answer:  112.5 
b8afd327-b836-4276-8556-eac89746b09f
 failure: -16105.0 good answer:  -6841 
6ba4045d-4222-4da0-a126-94b267a707ec
[Error]
 failure: [Error] good answer:  21.33 
e9b75381-50b1-4717-a322-31ed1eef4259
 Success: -2.0


In [76]:
def get_answer(llm, messages, table, q):
    values = table_convert.convert_multitable(table)    
    p, code = gen_code3(llm, messages, q, values)    
    #print(code)
    (v, s) = exec_code(code, values)
    return  (v, s)

In [77]:
from tqdm.notebook import tqdm as log_progress

devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
cnt = 0
res = []
for i, item in log_progress(devdf.iterrows()):
    if cnt == 36:
        break;
    #print(item['table']['uid'])
    table = item['table']['table']
    value_list = table_convert.convert_multitable(table)    
    
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']:   
            cnt = cnt + 1
            print(q['uid'])
            _table, _q =  get_question(q['uid'])
            
            table = _table['table']
            
            #[print(r) for r in table]
            
            q = _q['question']
            print(q, end='')
            
            
            r = []
            #for i in [0,1,2]:        
            #   r.append(get_answer(llm, messages, table, q))
            
            #if r[0][0] == r[1][0]:
            #    (v, s) = r[0]
            #elif r[1][0] == r[2][0]:
            #    (v, s) = r[1]
            #elif r[0][0] == r[2][0]:
            #    (v, s) = r[0]
            #else:
            #    (v, s) = (None, None)
                
            (v, s) = get_answer(llm, messages, table, q)    
            
            rng = 0.01
            if not isinstance(v, str)  and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
              print("\033[92m Success: " + str(v)+'\033[0m')
            else:    
                print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 
            
            
            (pred_value, pred_scale) = (v,s)
            if isinstance(pred_value, tuple) and len(pred_value) == 2:
                print('$$$$')
                (pred_value, pred_scale) = pred_value
            if pred_scale == "%" or pred_scale == "percentage"  :
                pred_scale = 'percent'
            if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                print('Invalid ', pred_scale)
                pred_scale = ""    
            #if item['table']['uid'] in res_scale and pred_scale != 'percent':
            #    pred_scale = res_scale[item['table']['uid']]
            err=""
            if isinstance(pred_value, str):
                #print("string")
                if  pred_value.startswith('[Error]'):
                    (pred_value, pred_scale) = ("", "")
                    err = v                                                        
            res.append(({"answer_type":_q["answer_type"], "answer": _q["answer"], 'scale': _q["scale"]}, pred_value, pred_scale, _q, code, "",value_list, err))

0it [00:00, ?it/s]

eb787966-fa02-401f-bfaf-ccabf3828b23
What is the change in Other in 2019 from 2018? Success: -12.6
05b670d3-5b19-438c-873f-9bf6de29c69e
What is the percentage change in Other in 2019 from 2018? Success: -22.22
b2786c1a-37de-4120-b03c-32bf5c81f157
What was the change in the amount for Appliances in 2019 from 2018? Success: -94.0
fe11f001-3bfe-4089-8108-412676f0a780
What was the percentage change in the amount for Appliances in 2019 from 2018? Success: -12.14
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf
What is the difference between the domestic and international discount rates as at September 30, 2019? Success: 2.1
4dc8be43-d8d9-4b08-9ffd-9c19012361ce
What is the year on year percentage change in domestic discount rate between 2018 and 2019? Success: 6.67
6c44a1a8-0785-43a0-90ab-7e21df2c57d9
What is the year on year percentage change in international expected return on plan assets between 2018 and 2019? Success: -8.11
a0414f81-8dc2-44b2-a441-2c9d9c805c4d
What is the 2019 average defined contri

In [78]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.7222222222222222 0.7222222222222222 0.7222222222222222


In [79]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
        continue
    
    #pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if ans['scale'] == pred_scale and  pred_scale == 'thousand':
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' == ' , ans['answer'], ans['scale'])
        
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #  if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if ans['answer'] != pred and ans['scale'] == pred_scale and pred_scale != "":
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #None
        print (q['uid'],q['derivation'], pred, pred_scale, ' <> ' , ans['answer'], ans['scale'])
        print(q['question'])
        #print(value_list)
        #print(value_list)
        #print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
print (good/len(res))

a983501d-2eec-486d-9661-e520c7c8af5e ((85,123 - 79,046) + (63,954 - 62,575)) / 2  2349.0   <>  3728 thousand
What was the average difference between EBITDA and underlying EBITDA for both FYs?
64c902c6-f426-4432-84b3-c10b3065716f -9,819 - 6,639  3180.0   <>  -16458 thousand
What was the difference in net profit between both FYs?
0.9444444444444444


In [22]:
len(res)

336

In [15]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if pred ==  ans['answer']*-1 and pred != 0:
        print(pred,ans['answer'])
        #if ans['scale'] != pred_scale:

-14212.0 14212
168.0 -168
1647.0 -1647
-1226114.0 1226114
43.0 -43
0.5 -0.5
0.5 -0.5
10.5 -10.5
-77395.0 77395
1859.0 -1859
10029.67 -10029.67
-1.9 1.9


In [21]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if (isinstance(pred, float) and pred/1000000 ==  ans['answer'])  and pred != 0 :
        print(pred,ans['answer'])